In [58]:
# Differential expression analysis

library(DESeq2)
library (pheatmap)
library(RColorBrewer)
library (ggplot2)
library(dplyr)

In [59]:
root="/Users/annasintsova/git_repos/spatial_dynamics_of_gene_expression_in_response_to_T6SS_attack/"
DATA=paste0(root, "tables/")
FIG=paste0(root, "figures/")

### Analyzing Differential Gene Expression between 0 and  30 min on the Dienes Line

In [61]:
counts_file = paste("/Users/annasintsova/git_repos/spatial_dynamics_of_gene_expression_in_response_to_T6SS_attack",
                     "/data/counts/stranded/2018-04-23_counts.csv", sep = "")
counts <- read.table(counts_file, row.names =1, sep = ",", header = TRUE)
experiment_info <- read.csv(paste0(root, "data/ref/study_design.csv"),
                                header = TRUE, row.names =1)
s_names <- c()
for (c in colnames(counts)){
    x <- gsub("X", "S", unlist(strsplit(c, "_"))[[1]])
    s_names <-c(s_names,x)
}
colnames(counts) <- s_names
exp_info_sample_names <- tibble::rownames_to_column(experiment_info, var = "rownames")

In [54]:
# Function to analyze differential gene expression based on TIME POINT

TimeAnalysisDE <- function (counts, exp_info, filename){
dds <- DESeqDataSetFromMatrix(countData = as.matrix(counts), colData = exp_info,
                                    design = ~time.point)
dds <- DESeq(dds)
res <- results(dds)

resSig <- subset(res, (padj < 0.1)& (log2FoldChange > 1 | log2FoldChange < -1))
write.csv(resSig[order(resSig$log2FoldChange, decreasing = TRUE),], paste0(DATA, filename),
            row.names = TRUE, quote = FALSE)

output <-  list("results" = resSig[order(resSig$log2FoldChange, decreasing = TRUE),], "dds"= dds)
return (output)
}

In [84]:
unlist(strsplit("2018-06-27-dienes-line-test.csv", "\\."))[1]

[1] "2018-06-27-dienes-line-test"

In [85]:
TimeAnalysis_subset <- function(exp_info_sample_names, counts, experiment_info, lane, pos, file_out){
    samples <- filter(exp_info_sample_names, lane.ID != lane & position == pos)[,1]
    counts <- counts[,samples]
    info <- experiment_info[samples,]
    analysis <- TimeAnalysisDE(counts, info, file_out)
    dds <- analysis$dds
    dds <- estimateSizeFactors(dds)
    norm_counts <- counts(dds, normalized=TRUE)
    write.csv(norm_counts, paste0(DATA, unlist(strsplit(file_out, "\\."))[1], "_counts.csv"))
    return(analysis)
}

In [86]:
dines_line_results <- TimeAnalysis_subset(exp_info_sample_names, counts, 
                                          experiment_info, "L1", "L-9C", "2018-06-27-dienes-line-test.csv")
dines_line_results$results

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 36 rows and 6 columns
                         baseMean log2FoldChange     lfcSE      stat
                        <numeric>      <numeric> <numeric> <numeric>
PMI1426                 2603.3788       2.536457 0.2963109  8.560121
PMI1425                 1440.3558       2.219405 0.3207280  6.919898
PMI2396                 2135.7921       2.217305 0.3405866  6.510251
PMI2397                  485.5849       1.802089 0.3317039  5.432823
PMI1427                 2649.0044       1.598964 0.3061397  5.222987
...                           ...            ...       ...       ...
PMI0648                1809.69278      -1.071900 0.2003107 -5.351189
PMI3401                 187.37700      -1.079391 0.2975396 -3.627722
PMI0729                 619.92438      -1.084232 0.3113269 -3.482616
PMI0861               12880.30164      -1.134173 0.2627818 -4.316025
fig|529507.6.peg.3417    40.27845      -1.37

In [56]:
Dienes_line_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "L-9C")[,1]
D_counts <- counts[,Dienes_line_samples]
D_exp_info <- experiment_info[Dienes_line_samples,]
D_exp_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S76079,L2,Case9,0'',Mix,L-9C,7.4
S76080,L2,Case9,0'',Mix,L-9C,7.3
S76081,L2,Case9,0'',Mix,L-9C,6.8
S76082,L2,Case10,30'',Mix,L-9C,6.2
S76083,L2,Case10,30'',Mix,L-9C,6.1
S76084,L2,Case10,30'',Mix,L-9C,5.5


In [57]:
Dienes_time <- TimeAnalysisDE(D_counts, D_exp_info, "2018-06-27-Dienes-line-time-no-quotes.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [19]:
Dienes_time$results

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 36 rows and 6 columns
                         baseMean log2FoldChange     lfcSE      stat
                        <numeric>      <numeric> <numeric> <numeric>
PMI1426                 2603.3788       2.536457 0.2963109  8.560121
PMI1425                 1440.3558       2.219405 0.3207280  6.919898
PMI2396                 2135.7921       2.217305 0.3405866  6.510251
PMI2397                  485.5849       1.802089 0.3317039  5.432823
PMI1427                 2649.0044       1.598964 0.3061397  5.222987
...                           ...            ...       ...       ...
PMI0648                1809.69278      -1.071900 0.2003107 -5.351189
PMI3401                 187.37700      -1.079391 0.2975396 -3.627722
PMI0729                 619.92438      -1.084232 0.3113269 -3.482616
PMI0861               12880.30164      -1.134173 0.2627818 -4.316025
fig|529507.6.peg.3417    40.27845      -1.37

### Analyzing Differential Gene Expression Behind the Merge at 0 and 30 min

In [20]:
WT_periphery_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "BH")[,1]
WT_periphery_counts <- counts[,WT_periphery_samples]
WT_periphery_exp_info <- experiment_info[WT_periphery_samples,]
WT_periphery_counts

,S76073,S76074,S76075,S76088,S76089,S76090
PMI0001,5071,5627,4484,3439,5636,6074
PMI0002,1547,1875,1452,1331,1935,2153
PMI0003,6233,5985,5254,5338,7894,8399
PMI0004,278,299,237,377,345,381
PMI0005,1462,989,884,628,1044,926
PMI0006,18064,16638,15093,13738,20981,20335
PMI0007,2792,3919,3356,2730,2342,2906
PMI0008,870,1033,931,1279,1216,1308
PMI0009,20533,24041,18494,23161,23117,19901
PMI0010,2381,3621,2653,4067,3135,2886


In [21]:
WT_periphery_time <- TimeAnalysisDE(WT_periphery_counts, WT_periphery_exp_info, "2018-06-27-WT-periphery-line-time.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [49]:
# 380 DE genes
WT_periphery_time$results
# get normalized counts:
dds <- WT_periphery_time$dds
dds <- estimateSizeFactors(dds)
norm_counts <- counts(dds, normalized=TRUE)

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 380 rows and 6 columns
                        baseMean log2FoldChange     lfcSE      stat
                       <numeric>      <numeric> <numeric> <numeric>
PMI0348                 788.2452       3.448874 0.3217504 10.719097
PMI0781                 412.3377       3.304738 0.3562953  9.275278
PMI3226                4790.0397       2.899292 0.3588041  8.080432
PMI1956                 167.1570       2.421253 0.3667462  6.601985
PMI1807                 883.3477       2.371982 0.2431928  9.753506
...                          ...            ...       ...       ...
PMI1343                588.78235      -1.883057 0.4193148 -4.490796
PMI3376               3370.81078      -2.038360 0.2292795 -8.890286
fig|529507.6.peg.1291   22.94104      -2.058813 0.4613959 -4.462141
PMI3598                 45.51311      -2.283743 0.3572940 -6.391776
PMI2149               1170.61275      -2.363668 0.35401

In [23]:
MUT_periphery_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "B9")[,1]
MUT_periphery_counts <- counts[,MUT_periphery_samples]
MUT_periphery_exp_info <- experiment_info[MUT_periphery_samples,]
MUT_periphery_counts

,S76076,S76077,S76078,S76085,S76086,S76087
PMI0001,3397,3732,4374,2375,4001,4212
PMI0002,1065,1353,1808,1058,1496,1624
PMI0003,4504,4394,5639,3552,5134,5832
PMI0004,207,241,326,450,279,324
PMI0005,1524,1209,1019,470,570,611
PMI0006,22358,17849,15837,12023,14745,15306
PMI0007,2588,3079,3872,2556,2259,2252
PMI0008,1024,877,1057,1410,1205,957
PMI0009,20130,29034,17750,24272,28713,24049
PMI0010,2350,3681,3218,4407,4159,3484


In [24]:
MUT_periphery_time <- TimeAnalysisDE(MUT_periphery_counts, MUT_periphery_exp_info, "2018-06-27-MUT-periphery-line-time.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [25]:
# 728 DE genes
MUT_periphery_time$results

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 728 rows and 6 columns
          baseMean log2FoldChange     lfcSE      stat       pvalue         padj
         <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
PMI1031  1118.3504       4.916655 0.3302088 14.889532 3.854608e-50 1.388815e-46
PMI0781   550.5741       4.700586 0.3798781 12.373931 3.616777e-35 6.515624e-32
PMI0348  1505.6800       4.600672 0.5028233  9.149680 5.710249e-20 2.571753e-17
PMI2408   120.5018       3.746797 0.4964789  7.546740 4.462876e-14 5.955459e-12
PMI1956   159.9439       3.744966 0.3915971  9.563313 1.140459e-21 8.218148e-19
...            ...            ...       ...       ...          ...          ...
PMI0913 3785.43253      -3.023720 0.7010666 -4.313029 1.610333e-05 1.534928e-04
PMI3598   50.08999      -3.272011 0.4342601 -7.534681 4.895316e-14 6.299223e-12
PMI0807 2977.62767      -3.285358 0.4683805 -7.014293 2.311142e-12 1.98

In [28]:
swarm_front_periphery_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "B")[,1]
swarm_front_periphery_counts <- counts[,swarm_front_periphery_samples]
swarm_front_periphery_exp_info <- experiment_info[swarm_front_periphery_samples,]
swarm_front_periphery_exp_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S77462,L3,Case13,0'',HI,B,NA
S77463,L3,Case13,0'',HI,B,NA
S77464,L3,Case13,0'',HI,B,NA
S77465,L3,Case14,30'',HI,B,NA
S77466,L3,Case14,30'',HI,B,NA
S77467,L3,Case14,30'',HI,B,NA


In [29]:
# 0 DE genes
swarm_front_periphery_time <- TimeAnalysisDE(swarm_front_periphery_counts, swarm_front_periphery_exp_info, 
                                             "2018-06-27-swarm-front-periphery-line-time.csv")
swarm_front_periphery_time$results

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 0 rows and 6 columns

### Analysis of Differential Gene Expression at the Swarm Front between 0 and 30 min

In [30]:
Swarm_line_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "L-HI")[,1]
Swarm_counts <- counts[,Swarm_line_samples]
Swarm_exp_info <- experiment_info[Swarm_line_samples,]
Swarm_exp_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S76067,L2,Case5,30'',HI,L-HI,6.8
S76068,L2,Case5,30'',HI,L-HI,6.1
S76069,L2,Case5,30'',HI,L-HI,6.1
S76070,L2,Case6,0'',HI,L-HI,6.4
S76071,L2,Case6,0'',HI,L-HI,4.5
S76072,L2,Case6,0'',HI,L-HI,6.1


In [31]:
Swarm_time <- TimeAnalysisDE(Swarm_counts, Swarm_exp_info, "2018-06-27-Swarm-line-time.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [32]:
Swarm_time$results

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 121 rows and 6 columns
         baseMean log2FoldChange     lfcSE      stat       pvalue         padj
        <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
PMI1031  1003.517       2.775656 0.2312591 12.002363 3.452952e-33 1.196448e-29
PMI0437  5635.458       2.063356 0.2094392  9.851812 6.731923e-23 5.831528e-20
PMI3038  1196.089       2.044178 0.2278525  8.971497 2.925126e-19 1.013556e-16
PMI2847  4752.583       1.912011 0.1969639  9.707418 2.803483e-22 1.825954e-19
PMI2254 10780.565       1.862599 0.1613418 11.544432 7.875990e-31 1.364515e-27
...           ...            ...       ...       ...          ...          ...
PMI1426 1939.4264      -1.631109 0.2685475 -6.073820 1.249028e-09 6.658279e-08
PMI0176  219.7845      -1.854880 0.2793990 -6.638824 3.161964e-11 2.235960e-09
PMI1425 1318.4807      -1.966218 0.2666194 -7.374625 1.648088e-13 1.730492e-11
PM

### Analysis of Differential Gene Expression at the Dienes Line/Swarm Front at 0 and 4 hours

In [34]:
counts_file = paste("/Users/annasintsova/git_repos/spatial_dynamics_of_gene_expression_in_response_to_T6SS_attack",
                     "/data/counts/reverse/2018-04-25_counts.csv", sep = "")
counts <- read.table(counts_file, row.names =1, sep = ",", header = TRUE)
s_names <- c()
for (c in colnames(counts)){
    x <- gsub("X", "S", unlist(strsplit(c, "_"))[[1]])
    s_names <-c(s_names,x)
}
colnames(counts) <- s_names
counts

,S77464,S76068,S76082,S63632,S76076,S77466,S63638,S76088,S76074,S76080,⋯,S63639,S76089,S76087,S76073,S76079,S77463,S76067,S76071,S63635,S76085
PMI0001,42,51,90,9192,37,54,9655,36,46,44,⋯,9885,38,59,52,56,37,53,39,7130,24
PMI0002,17,27,57,5492,20,28,5068,26,31,31,⋯,5505,29,24,30,34,19,31,33,4140,18
PMI0003,96,50,118,8543,80,126,7497,78,89,82,⋯,7840,93,91,104,83,133,89,84,7381,55
PMI0004,71,82,105,247,120,57,170,78,112,54,⋯,206,65,66,73,70,40,69,137,353,93
PMI0005,12,25,26,1152,22,17,905,7,17,12,⋯,1920,18,23,22,12,6,19,24,1624,15
PMI0006,164,223,270,27659,204,169,31052,138,161,189,⋯,28434,156,174,192,173,115,215,177,21339,105
PMI0007,14,21,13,7766,11,17,6672,17,23,15,⋯,6466,14,17,16,21,4,21,9,5976,14
PMI0008,14,12,8,3069,7,6,2390,7,7,10,⋯,3385,7,7,1,10,9,8,8,3660,12
PMI0009,48,274,463,94391,137,62,115046,182,153,386,⋯,77298,138,217,149,329,98,331,148,37922,172
PMI0010,10,37,98,26743,19,14,27115,29,29,82,⋯,21866,32,40,21,79,10,74,36,13490,60


In [35]:
Swarm_line_samples <- filter(exp_info_sample_names, lane.ID != "L2" & position == "L-HI")[,1]
Swarm_counts <- counts[,Swarm_line_samples]
Swarm_exp_info <- experiment_info[Swarm_line_samples,]
Swarm_exp_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S63630,L1,Case1,0'',HI,L-HI,NA
S63631,L1,Case1,0'',HI,L-HI,NA
S63632,L1,Case1,0'',HI,L-HI,NA
S63633,L1,Case2,4',HI,L-HI,NA
S63634,L1,Case2,4',HI,L-HI,NA
S63635,L1,Case2,4',HI,L-HI,NA


In [36]:
Swarm_time <- TimeAnalysisDE(Swarm_counts, Swarm_exp_info, "2018-06-27-Swarm-line-4-hours-time.csv")
Swarm_time$results

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


log2 fold change (MAP): time.point 4' vs 0'' 
Wald test p-value: time.point 4' vs 0'' 
DataFrame with 1117 rows and 6 columns
          baseMean log2FoldChange     lfcSE       stat        pvalue
         <numeric>      <numeric> <numeric>  <numeric>     <numeric>
PMI1956  1201.5699       6.350921 0.2401233  26.448580 3.788618e-154
PMI3226 13610.7365       6.048924 0.2222813  27.212929 4.566639e-163
PMI2662 29614.6336       5.713470 0.6489596   8.804045  1.319710e-18
PMI0781  3204.5043       5.619931 0.2612521  21.511524 1.214474e-102
PMI2442   351.6271       5.613313 0.7590562   7.395122  1.412785e-13
...            ...            ...       ...        ...           ...
PMI2246  1809.6040      -3.510083 0.2927591 -11.989664  4.025339e-33
PMI2250  2682.1117      -3.537755 0.5075118  -6.970784  3.151803e-12
PMI2149 10419.3361      -3.672390 0.2281109 -16.099147  2.586461e-58
PMI2899   654.0655      -3.708357 0.2567857 -14.441448  2.838166e-47
PMI2898 10997.9384      -4.284880 0.5460703  -

In [37]:
Dienes_line_samples <- filter(exp_info_sample_names, lane.ID != "L2" & position == "L-9C")[,1]
Dienes_counts <- counts[,Dienes_line_samples]
Dienes_exp_info <- experiment_info[Dienes_line_samples,]
Dienes_exp_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S63636,L1,Case3,0'',Mix,L-9C,NA
S63638,L1,Case3,0'',Mix,L-9C,NA
S63639,L1,Case4,4',Mix,L-9C,NA
S63641,L1,Case4,4',Mix,L-9C,NA


In [39]:
Dienes_time <- TimeAnalysisDE(Dienes_counts, Dienes_exp_info, "2018-06-27-Dienes-line-4-hours-time.csv")
Dienes_time$results

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


log2 fold change (MAP): time.point 4' vs 0'' 
Wald test p-value: time.point 4' vs 0'' 
DataFrame with 899 rows and 6 columns
         baseMean log2FoldChange     lfcSE      stat       pvalue         padj
        <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
PMI1956   962.718       5.881001 0.7169367  8.202958 2.345440e-16 2.064574e-13
PMI3002   909.312       5.026773 0.5122344  9.813423 9.856694e-23 3.470542e-19
PMI1804  7415.958       4.865792 0.7069502  6.882793 5.869045e-12 1.589608e-09
PMI3226 10516.382       4.752490 0.7147730  6.648950 2.951919e-11 6.496066e-09
PMI3699 10944.861       4.743317 0.5057246  9.379250 6.644347e-21 1.169737e-17
...           ...            ...       ...       ...          ...          ...
PMI3705  576.2815      -3.759796 0.8725690 -4.308881 1.640826e-05 5.146401e-04
PMI0807  475.2360      -3.817443 0.8811474 -4.332354 1.475232e-05 4.722083e-04
PMI3706 3155.8502      -3.881818 0.8698797 -4.462476 8.101784e-06 2.971498e-04
PMI017